### Numerical Greeks Calculation

In [1]:
from QuantLib import *

In [2]:
today = Date(8, October, 2014)
Settings.instance().evaluationDate = today

In [4]:
# Knock-in barrier option
'''
BarrierOption reference Option(barrierType, barrier, rebate,
                                      payoff, exercise);
                                                                            
'''

option = BarrierOption(Barrier.UpIn,
                       120.0,         # barrier
                       0.0,           # rebate
                       PlainVanillaPayoff(Option.Call, 100.0),
                       EuropeanExercise(Date(8, January, 2015)))

In [5]:
u = SimpleQuote(100.0)
r = SimpleQuote(0.01)
sigma = SimpleQuote(0.20)

In [6]:
'''
DayCounter dayCounter = Actual365Fixed();
Handle<Quote> h1(riskFreeRate);
Handle<Quote> h2(volatility);
Handle<YieldTermStructure> flatRate(
                                    ext::shared_ptr<YieldTermStructure>(
                                    new FlatForward(0, NullCalendar(),
                                    h1, dayCounter)));
Handle<BlackVolTermStructure> flatVol(
                                    ext::shared_ptr<BlackVolTermStructure>(
                                    new BlackConstantVol(0, NullCalendar(),
                                    h2, dayCounter)));


'''



riskFreeCurve = FlatForward(0, TARGET(), QuoteHandle(r), Actual360())
volatility = BlackConstantVol(0, TARGET(), QuoteHandle(sigma), Actual360())

In [7]:
process = BlackScholesProcess(QuoteHandle(u),
                              YieldTermStructureHandle(riskFreeCurve),
                              BlackVolTermStructureHandle(volatility))

In [8]:
option.setPricingEngine(AnalyticBarrierEngine(process))

In [9]:
print(option.NPV())

1.3657980739109867


### There is no Greeks, e.g Delta

Using numerical differentiation to approximate the Greeks

Delta = (P(u0 + h) - P(u0 - h))/2h  
Gamma = (P(u0 + h) - 2 * P(u0) + P(u0 - h)) / square(h)

In [10]:
u0 = u.value()
h  = 0.01

In [11]:
P0 = option.NPV()
print(P0)

1.3657980739109867


In [12]:
u.setValue(u0 + h)
P_plus = option.NPV()
print(P_plus)

1.3688112201958083


In [13]:
u.setValue(u0 - h)
P_minus = option.NPV()
print(P_minus)

1.3627900998610207


In [14]:
u.setValue(u0)

In [15]:
# Delta and Gamma Calculation

Delta = (P_plus - P_minus) / (2 * h)
Gamma = (P_plus - 2 * P0 + P_minus) / (h * h)

print(Delta)
print(Gamma)

0.3010560167393761
0.05172234855521651


In [17]:
# Rho Calculation

r0 = r.value()
h  = 0.0001
r.setValue(r0 + h)
P_plus = option.NPV()
r.setValue(r0)
Rho = (P_plus - P0) / h

print(Rho)

6.531038494277386


In [18]:
# Vega Calculation

sigma0 = sigma.value()
h = 0.0001
sigma.setValue(sigma0 + h)
P_plus = option.NPV()
sigma.setValue(sigma0)
Vega = (P_plus - P0) / h

print(Vega)

26.52519924198904


In [19]:
# Theta Calculation

Settings.instance().evaluationDate = today + 1
P1 = option.NPV()
h  = 1/365

Theta = (P1 - P0) / h

print(Theta)

-10.770888399441302
